# Synthetic Experiments

## Sample synthetic data

In [ ]:
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
import numpy as np

import choice_learn
from python.data import SyntheticDataGenerator
from choice_learn.basket_models import Trip, TripDataset

## Sample purchased baskets

In [ ]:
items_nests = {0:[0, 1, 2],
1: [3, 4, 5],
2: [6],
3: [7]}

nests_interactions = [["", "compl", "neutral", "neutral"],
["compl", "", "neutral", "neutral"],
["neutral", "neutral", "", "neutral"],
["neutral", "neutral", "neutral", ""]]

data_gen = SyntheticDataGenerator(items_nest=items_nests, nests_interactions=nests_interactions)

In [ ]:
dataset = data_gen.generate_dataset(n_baskets=10000)

In [ ]:
trip_list = []
for basket in dataset:
    trip_list.append(Trip(purchases=basket, prices=np.zeros((8, )), assortment=0))

trip_dataset = TripDataset(trips=trip_list, available_items=np.ones((1, 8)))

## Modelling 

In [ ]:
from choice_learn.basket_models import AleaCarta, Shopper

In [ ]:
latent_sizes = {"preferences": 6, "price": 3, "season": 3}
n_negative_samples = 2
optimizer = "adam"
lr = 5e-4
epochs = 150
batch_size = 32

aleacarta = AleaCarta(
    item_intercept=False,
    price_effects=False,
    seasonal_effects=False,
    latent_sizes=latent_sizes,
    n_negative_samples=n_negative_samples,
    optimizer=optimizer,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
)

aleacarta.instantiate(n_items=8, n_stores=2)

In [ ]:
history = aleacarta.fit(trip_dataset)

In [ ]:
plt.plot(history["train_loss"])
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.show()

In [ ]:
for batch in shopper_trip_dataset.iter_batch(batch_size=4):
    break

In [ ]:
latent_sizes = {"preferences": 6, "price": 3, "season": 3}
n_negative_samples = 2
optimizer = "adam"
lr = 5e-4
epochs = 150
batch_size = 32

trip_list = []
for basket in dataset:
    trip_list.append(Trip(purchases=np.array(basket+[-1])+1, prices=np.zeros((9, )), assortment=0))

shopper_trip_dataset = TripDataset(trips=trip_list, available_items=np.ones((1, 9)))

shopper = Shopper(
    item_intercept=False,
    price_effects=False,
    seasonal_effects=False,
    think_ahead=False,
    latent_sizes=latent_sizes,
    n_negative_samples=n_negative_samples,
    optimizer=optimizer,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
)

shopper.instantiate(n_items=9, n_stores=1)
print(shopper.trainable_weights[-1])
history = shopper.fit(shopper_trip_dataset)
print(shopper.trainable_weights[-1])
plt.plot(history["train_loss"])
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.show()

## Results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np


mask = np.ones((8,8), dtype=bool)
alea_results = []
shopper_results = []
for i in range(8):
        r_1 = aleacarta.compute_batch_utility(item_batch=np.array(list(range(8))),
                basket_batch=np.array([[i] for _ in range(8)]) ,
                store_batch=np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                week_batch=np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                price_batch=np.array([[0, 0, 0, 0, 0, 0] for _ in range(8)]))

        r_2 = shopper.compute_batch_utility(item_batch=np.array(list(range(1, 9))),
                basket_batch=np.array([[i+1] for _ in range(8)]) ,
                available_item_batch=np.ones((9, 1)),
                store_batch=np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                week_batch=np.array([0, 0, 0, 0, 0, 0, 0, 0]),
                price_batch=np.array([[0, 0, 0, 0, 0, 0] for _ in range(8)]))
        m = np.ones(8)
        m[i] = 0
        den = np.exp(r_1) * m
        r_1 = den / den.sum()
        den = np.exp(r_2) * m
        r_2 = den / den.sum()

        alea_results.append(r_1)
        shopper_results.append(r_2)
        mask[i][i] = False

alea_results = np.stack(alea_results)
shopper_results = np.stack(shopper_results)
mask = np.ma.masked_where(mask, alea_results)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

for ax_, model in enumerate([aleacarta, shopper]):

        axes[ax_].set_xticks([], [])
        axes[ax_].set_yticks([], [])
        im = axes[ax_].imshow((alea_results, shopper_results)[ax_], cmap="Spectral", alpha=0.99, vmin=0, vmax=1)
        axes[ax_].imshow(mask, cmap=mpl.colors.ListedColormap(['white']), alpha=1)
        axes[ax_].set_title(f"{model}")

cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.69])
fig.colorbar(im, cax=cbar_ax)
fig.suptitle("Estimated Conditional Probabilities")